In [1]:
# -*- coding: utf-8 -*-
    
import numpy as np
import pandas as pd

## Чтение данных

Распаковываем архив:

In [2]:
! unrar e -r feedback.csv.rar


UNRAR 5.30 beta 2 freeware      Copyright (c) 1993-2015 Alexander Roshal


Extracting from feedback.csv.rar

Extracting  X_train.csv                                                   10% 20% 31% 41% 51% 62% 72% 83% 93% 99%  OK 
All OK


Значения в X_train.csv разделены точкой с запятой (';'). Этот символ также встречается и в комментариях, так что приходится парсить данные руками:

In [3]:
def remove_double_quotes(s):
    if not len(s):
        return s
    
    if not '"' in s:
        return s
    
    first, last = 0, len(s) - 1
    while s[first] != '"':
        first += 1
    while s[last] != '"':
        last -= 1
        
    return s[first+1:last]


def safe_conversion(val):
    try:
        val = int(val)
    except:
        pass
    
    try:
        val = float(val)
    except:
        pass
    
    return val


def read_data(filename):
    
    with open(filename, 'r') as f:
        lines = f.readlines()
        columns = map(remove_double_quotes, lines[0].split(';'))
        X = pd.DataFrame(columns=columns)
        
        for line in lines[1:]:
            values = line.split(';')
            values = values[:8] + [';'.join(values[8:-2])] + values[-2:]

            row = pd.DataFrame(data=np.array(values).reshape(1, -1), columns=columns)
            X = pd.concat((X, row), axis=0)  
    
    for column in X.columns:
        X[column] = X[column].apply(lambda x: safe_conversion(remove_double_quotes(x)))

    X.comment = X.comment + X.commentNegative + X.commentPositive
    
    X = X[['userName', 'reting', 'comment']]
    X = X.set_index(np.arange(X.shape[0]))
    X.comment = X.comment.apply(lambda x: (x.decode('cp1251').encode('utf8')))
    
    return X

In [4]:
X = read_data('X_train.csv')
X.head()

,userName,reting,comment
0,b2898a81b45310b30beb8fc0c0a9ce1e,2.0,"2,5 года работала и все...устала! Лампочка гор..."
1,538c73d64461e13907bb95c51c38bfbc,2.0,Через 2 месяца после истечении гарантийного ср...
2,ddca2d0101513a6209db7868eed8be05,4.0,пользуюсь уже три недели. нареканий ни каких н...
3,289c20015b3713a82ba5ddf774d996f7,5.0,Ребят этот системный блок подойдёт для игры кс...
4,5576f82d149d4f688644fef2322c63ef,5.0,"я считаю, что яри замечательный телефон! Прият..."


## Предобработка комментариев:

NLTK (Natural Language ToolKit) -- инструмент для работы с естественным языком:

In [5]:
import nltk

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

Стемминг -- выделение значащей части слова ( https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%B5%D0%BC%D0%BC%D0%B8%D0%BD%D0%B3 ) :

In [6]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('russian')

Стоп-слова:

In [7]:
from nltk.corpus import stopwords

stopwords = stopwords.words('russian')

for word in stopwords[:10]:
    print word.encode('utf8')

и
в
во
не
что
он
на
я
с
со


PyMorphy2 -- инструмент для работы с текстами на русском языке ( https://pymorphy2.readthedocs.io/en/latest/ ):

In [8]:
#import pymorphy2

#morph = pymorphy2.MorphAnalyzer()

Параметр **clear** отвечает за фильтрацию комментариев:
* **none** -- "как есть"
* **pymorphy** -- оставить только существительные, прилагательные, глаголы, наречия, причастия и деепричастия
* **stopwords** -- удалить стоп-слова

Параметр **stem** отвечает за преобразование слов:
* **none** -- "как есть"
* **pymorphy** -- привести к начальной форме
* **stem** -- стемминг

In [9]:
import re

def preprocess_single_comment(text, stem='none', clear='none'):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        token = token.lower()      
        token = re.sub(r',-.:;?1//\|',' ', token)
        for word_part in token.split(' '):
            if re.compile(u'^[а-яА-Я]+$').match(word_part):
                filtered_tokens.append(word_part)   
                
    if clear == 'pymorphy':
        meaningful_pos = ['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'ADVB']
        filtered_tokens = [t for t in filtered_tokens if morph.parse(t)[0].tag.POS in meaningful_pos]
    
    elif clear == 'stopwords':
        filtered_tokens = [t for t in filtered_tokens if t not in stopwords]
    
    if stem == 'stem':
        filtered_tokens = [stemmer.stem(t) for t in filtered_tokens]
    
    elif stem == 'pymorphy':
        filtered_tokens = [morph.parse(t)[0].normal_form for t in filtered_tokens]
     
    return filtered_tokens


def preprocess(X, stem='none', clear='none'):
    X_preprocessed = pd.DataFrame.copy(X)
    for ix in X.index:
        comment = X.ix[ix, 'comment'].decode('utf8')
        comment_stemmed = preprocess_single_comment(comment, stem=stem, clear=clear)
        if stem == 'mystem':
            X_preprocessed.ix[ix, 'comment'] = ''.join(comment_stemmed).encode('utf8')
        else:
            X_preprocessed.ix[ix, 'comment'] = ' '.join(comment_stemmed)
        
    return X_preprocessed

Удаляем стоп-слова, применяем стемминг:

In [10]:
X_preprocessed = preprocess(X, stem='stem', clear='stopwords')
X_preprocessed.head()

,userName,reting,comment
0,b2898a81b45310b30beb8fc0c0a9ce1e,2.0,год работа уста лампочк гор
1,538c73d64461e13907bb95c51c38bfbc,2.0,месяц истечен гарантийн срок машинк навернул н...
2,ddca2d0101513a6209db7868eed8be05,4.0,польз недел нарекан как положительн эмоц вчер ...
3,289c20015b3713a82ba5ddf774d996f7,5.0,реб системн блок игр кс го средн гастройк
4,5576f82d149d4f688644fef2322c63ef,5.0,счита яр замечательн телефон приятн держа рук ...


## Разбиение данных на обучение-тест

In [11]:
from sklearn.model_selection import train_test_split

np.random.seed(42)

def split_data(X, mode='folds', k=5, test_size=0.9):
    users = np.unique(X.userName)

    if mode == 'folds':
        np.random.shuffle(users)
        folds = []
        for i in range(k - 1):
            fold_users, users = train_test_split(users, train_size=1.0 / (k - i))
            folds.append(fold_users)
        
        folds.append(users)
        return folds
    
    elif mode == 'train_test':
        train_users, test_users = train_test_split(users, test_size=test_size)
        return train_users, test_users
    
    else:
        raise NotImplemented

## Формирование признакового пространства

https://ru.wikipedia.org/wiki/TF-IDF

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

def fit(X):
    return TfidfVectorizer(min_df=1).fit(X.comment)

def transform(X, model):
    return model.transform(X.comment)

## Предсказание

Модель -- линейная регрессия с L1-, L2-регуляризаторами:

In [13]:
from sklearn.linear_model import ElasticNet

clf = ElasticNet(alpha=0.001, l1_ratio=0.01)

Метрика -- RMSE:

In [14]:
from sklearn.metrics import mean_squared_error
import itertools

def _rmse(y, p):
    return np.sqrt(mean_squared_error(y, p))


def calculate_rmse(X, train_users, test_users, clf, fit, transform):
    rmse_scores = []
    X_train = X[X.userName.isin(train_users)]
    X_test = X[X.userName.isin(test_users)]
        
    y_train = np.array(X_train.reting, dtype=float)
    y_test = np.array(X_test.reting, dtype=float)
        
    model = fit(X_train)
    X_train = transform(X_train, model)
    X_test = transform(X_test, model)
        
    p_test = clf.fit(X_train, y_train).predict(X_test)
       
    return _rmse(y_test, p_test)


def calculate_fold_average_rmse(X, folds, clf, fit, transform, verbose=False):
    rmse_scores = []
    for i in range(len(folds)):
        train_users = list(itertools.chain.from_iterable([folds[j] for j in range(len(folds)) if j != i]))
        test_users = folds[i]
        
        rmse_scores.append(calculate_rmse(X, train_users, test_users, clf, fit, transform))
        
        if verbose:
            print 'FOLD {}\t RMSE: {}\n'.format(i + 1, rmse_scores[-1])
    
    return np.mean(rmse_scores)

Средняя ошибка по 10 фолдам:

In [15]:
folds = split_data(X_preprocessed, mode='folds', k=10)
average_rmse = calculate_fold_average_rmse(X_preprocessed, folds, clf, fit, transform, verbose=True)
print 'AVERAGE RMSE: ', average_rmse

FOLD 1	 RMSE: 0.993650068809

FOLD 2	 RMSE: 1.12919749226

FOLD 3	 RMSE: 1.12381967861

FOLD 4	 RMSE: 1.0540862892

FOLD 5	 RMSE: 1.14778359076

FOLD 6	 RMSE: 1.17630882632

FOLD 7	 RMSE: 1.02487589553

FOLD 8	 RMSE: 1.15267520816

FOLD 9	 RMSE: 1.05621716411

FOLD 10	 RMSE: 0.958681359376

AVERAGE RMSE:  1.08172955731


Ошибка на тесте:

In [16]:
train_users, test_users = split_data(X_preprocessed, mode='train_test', test_size=0.1)
print 'RMSE: ', calculate_rmse(X_preprocessed, train_users, test_users, clf, fit, transform)

RMSE:  0.98807363042
